# Análise de Decisão

Pense Bayes, Segunda Edição

Copyright 2020 Allen B. Downey

Licença: [Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/)

In [1]:
# If we're running on Colab, install empiricaldist
# https://pypi.org/project/empiricaldist/

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install empiricaldist

In [2]:
# Get utils.py

from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)
    
download('https://github.com/AllenDowney/ThinkBayes2/raw/master/soln/utils.py')

In [3]:
from utils import set_pyplot_params
set_pyplot_params()

Este capítulo apresenta um problema inspirado no programa de jogos *O preço está certo*.

É um exemplo parvo, mas demonstra um processo útil chamado Bayesian [decision analysis](https://en.wikipedia.org/wiki/Decision_analysis).

Como em exemplos anteriores, utilizaremos dados e distribuição prévia para calcular uma distribuição posterior; depois utilizaremos a distribuição posterior para escolher uma estratégia óptima num jogo que envolva licitação.

Como parte da solução, utilizaremos a estimativa da densidade do núcleo (KDE) para estimar a distribuição prévia, e uma distribuição normal para calcular a probabilidade dos dados.

E no final do capítulo, coloco um problema relacionado que pode resolver como um exercício.

## O preço é o problema certo

A 1 de Novembro de 2007, os concorrentes Letia e Nathaniel apareceram no *The Price is Right*, um programa de jogos de televisão americano. Concorreram num jogo chamado "The Showcase", onde o objectivo é adivinhar o preço de uma colecção de prémios. O concorrente que se aproximar do preço real, sem passar por cima, ganha os prémios.

Nathaniel foi o primeiro. A sua vitrina incluía uma máquina de lavar loiça, uma garrafeira, um computador portátil e um carro. Ele ofereceu 26.000 dólares.

A vitrina de Letia incluía uma máquina de pinball, um jogo de vídeo arcade, uma mesa de bilhar, e um cruzeiro pelas Bahamas. Ela ofereceu {\i1}21.500$.

O preço real da vitrina de Nathaniel era de 25.347 dólares. A sua oferta era demasiado alta, por isso perdeu.

O preço real da montra de Letia foi de 21.578 dólares. 

Ela só ficou de fora por 78 dólares, por isso ganhou a sua montra e, como a sua oferta ficou de fora por menos de 250, também ganhou a montra de Nathaniel.

Para um pensador Bayesiano, este cenário sugere várias questões:

1. Antes de ver os prémios, que crenças anteriores devem os concorrentes ter sobre o preço da montra?

2. Depois de ver os prémios, como devem os concorrentes actualizar essas crenças?

3. Com base na distribuição posterior, o que devem os concorrentes licitar?

A terceira questão demonstra uma utilização comum dos métodos Bayesianos: análise de decisão.

Este problema é inspirado por [an example](https://nbviewer.jupyter.org/github/CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/blob/master/Chapter5_LossFunctions/Ch5_LossFunctions_PyMC3.ipynb) no livro de Cameron Davidson-Pilon, [*Probablistic Programming and Bayesian Methods for Hackers*](http://camdavidsonpilon.github.io/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers).

## O Prior

Para escolher uma distribuição prévia de preços, podemos tirar partido de dados de episódios anteriores. Felizmente, [fans of the show keep detailed records](https://web.archive.org/web/20121107204942/http://www.tpirsummaries.8m.com/). 

Para este exemplo, descarreguei ficheiros contendo o preço de cada montra das épocas de 2011 e 2012 e as licitações oferecidas pelos concorrentes.

As células seguintes carregam os ficheiros de dados.

In [4]:
# Load the data files

download('https://raw.githubusercontent.com/AllenDowney/ThinkBayes2/master/data/showcases.2011.csv')
download('https://raw.githubusercontent.com/AllenDowney/ThinkBayes2/master/data/showcases.2012.csv')

A função seguinte lê os dados e limpa-os um pouco.

In [5]:
import pandas as pd

def read_data(filename):
    """Read the showcase price data."""
    df = pd.read_csv(filename, index_col=0, skiprows=[1])
    return df.dropna().transpose()

Vou ler os dois ficheiros e concatená-los.

In [6]:
df2011 = read_data('showcases.2011.csv')
df2012 = read_data('showcases.2012.csv')

df = pd.concat([df2011, df2012], ignore_index=True)

In [7]:
print(df2011.shape, df2012.shape, df.shape)

Aqui está como é o conjunto de dados:

In [8]:
df.head(3)

As duas primeiras colunas, `Showcase 1` e `Showcase 2`, são os valores das vitrinas em dólares.

As duas colunas seguintes são as licitações feitas pelos concorrentes.

As duas últimas colunas são as diferenças entre os valores reais e as licitações.

## Estimativa da Densidade do Núcleo

Este conjunto de dados contém os preços de 313 mostruários anteriores, que podemos considerar como uma amostra da população de preços possíveis.

Podemos utilizar esta amostra para estimar a distribuição prévia de preços de montras.  Uma forma de o fazer é a estimativa da densidade do grão (KDE), que utiliza a amostra para estimar uma distribuição suave.  Se não estiver familiarizado com o KDE, pode [read about it here](https://mathisonian.github.io/kde).

SciPy fornece `gaussian_kde`, que recolhe uma amostra e devolve um objecto que representa a distribuição estimada.

A seguinte função toma "amostra", faz um KDE, avalia-o a uma dada sequência de quantidades, "qs", e devolve o resultado como um PMF normalizado.

In [9]:
from scipy.stats import gaussian_kde
from empiricaldist import Pmf

def kde_from_sample(sample, qs):
    """Make a kernel density estimate from a sample."""
    kde = gaussian_kde(sample)
    ps = kde(qs)
    pmf = Pmf(ps, qs)
    pmf.normalize()
    return pmf

Podemos utilizá-lo para estimar a distribuição de valores para o Showcase 1:

In [10]:
import numpy as np

qs = np.linspace(0, 80000, 81)
prior1 = kde_from_sample(df['Showcase 1'], qs)

Aqui está o que parece:

In [11]:
from utils import decorate

def decorate_value(title=''):
    decorate(xlabel='Showcase value ($)',
        ylabel='PMF',
        title=title)

In [12]:
prior1.plot(label='Prior 1')
decorate_value('Prior distribution of showcase value')

**Exercício:** Utilizar esta função para fazer um `Pmf` que representa a distribuição prévia para o Showcase 2, e plotá-lo.

In [13]:
# Solution goes here

In [14]:
# Solution goes here

## Distribuição do erro

Para actualizar estes antecedentes, temos de responder a estas questões:

* Que dados devemos considerar e como devemos quantificá-los?

* Podemos calcular uma função de probabilidade; isto é, para cada preço hipotético, podemos calcular a probabilidade condicional dos dados?

Para responder a estas perguntas, vou modelar cada concorrente como um instrumento de orientação de preços com características de erro conhecidas. 

Neste modelo, quando o concorrente vê os prémios, adivinha o preço de cada prémio e adiciona os preços.

Chamemos a isto "adivinhação" total.

Agora a pergunta que temos de responder é: "Se o preço real é "preço", qual é a probabilidade de o palpite do concorrente ser "palpite"?

Da mesma forma, se definirmos "erro = palpite - preço", podemos perguntar: "Qual é a probabilidade de o palpite do concorrente estar errado por "erro"?

Para responder a esta pergunta, vou utilizar novamente os dados históricos. 

Para cada mostruário do conjunto de dados, vejamos a diferença entre a oferta do concorrente e o preço real:

In [15]:
sample_diff1 = df['Bid 1'] - df['Showcase 1']
sample_diff2 = df['Bid 2'] - df['Showcase 2']

Para visualizar a distribuição destas diferenças, podemos utilizar novamente o KDE.

In [16]:
qs = np.linspace(-40000, 20000, 61)
kde_diff1 = kde_from_sample(sample_diff1, qs)
kde_diff2 = kde_from_sample(sample_diff2, qs)

Eis como são estas distribuições:

In [17]:
kde_diff1.plot(label='Diff 1', color='C8')
kde_diff2.plot(label='Diff 2', color='C4')

decorate(xlabel='Difference in value ($)',
        ylabel='PMF',
        title='Difference between bid and actual value')

Parece que as propostas são demasiado baixas mais frequentemente do que demasiado altas, o que faz sentido.  Lembre-se de que, segundo as regras do jogo, perde-se se se apresentar uma proposta excessiva, pelo que os concorrentes provavelmente apresentam uma proposta inferior em certa medida deliberadamente.

Por exemplo, se eles adivinharem que o valor da vitrina é de 40.000 dólares, podem licitar 36.000 dólares para evitar passar por cima.

Parece que estas distribuições são bem modeladas por uma distribuição normal, pelo que podemos resumi-las com a sua média e desvio padrão.

Por exemplo, aqui está a média e o desvio padrão de `Diff` para o Jogador 1.

In [18]:
mean_diff1 = sample_diff1.mean()
std_diff1 = sample_diff1.std()

print(mean_diff1, std_diff1)

Agora podemos usar estas diferenças para modelar a distribuição de erros do concorrente.

Este passo é um pouco complicado porque na realidade não conhecemos os palpites do concorrente; apenas sabemos o que ele licita.

Por isso, temos de fazer algumas suposições:

* Presumo que os concorrentes subestimam a sua proposta porque estão a ser estratégicos, e que em média os seus palpites são exactos.  Por outras palavras, a média dos seus erros é 0.

* Mas suponho que a propagação das diferenças reflecte a verdadeira propagação dos seus erros.  Assim, vou utilizar o desvio padrão das diferenças como o desvio padrão dos seus erros.

Com base nestes pressupostos, farei uma distribuição normal com parâmetros 0 e `std_diff1`.

SciPy fornece um objecto chamado `norm` que representa uma distribuição normal com a média e o desvio padrão dados.

In [19]:
from scipy.stats import norm

error_dist1 = norm(0, std_diff1)

O resultado é um objecto que fornece `pdf`, que avalia a função de densidade de probabilidade da distribuição normal.

Por exemplo, aqui está a densidade de probabilidade de `error=-100`, com base na distribuição de erros para o Jogador 1.

In [20]:
error = -100
error_dist1.pdf(error)

Por si só, este número não significa muito, porque as densidades de probabilidade não são probabilidades. Mas são proporcionais às probabilidades, pelo que podemos usá-las como probabilidades numa actualização Bayesiana, como veremos na secção seguinte.

## Actualização

Suponha que é o Jogador 1.  Vê os prémios na sua vitrina e o seu palpite para o preço total é de 23.000 dólares.

Do vosso palpite subtrairei cada preço hipotético na distribuição prévia; o resultado é o vosso erro sob cada hipótese.

In [21]:
guess1 = 23000
error1 = guess1 - prior1.qs

Agora suponha que sabemos, com base no desempenho passado, que o seu erro de estimativa é bem modelado por `error_dist1`.

Sob essa hipótese, podemos calcular a probabilidade do seu erro sob cada hipótese.

In [22]:
likelihood1 = error_dist1.pdf(error1)

O resultado é um conjunto de probabilidades, que podemos utilizar para actualizar o anterior.

In [23]:
posterior1 = prior1 * likelihood1
posterior1.normalize()

Eis como se parece a distribuição posterior:

In [24]:
prior1.plot(color='C5', label='Prior 1')
posterior1.plot(color='C4', label='Posterior 1')

decorate_value('Prior and posterior distribution of showcase value')

Como o seu palpite inicial está no extremo inferior da gama, a distribuição posterior deslocou-se para a esquerda.  Podemos calcular a média posterior para ver por quanto.

In [25]:
prior1.mean(), posterior1.mean()

Antes de ver os prémios, esperava ver uma vitrina com um valor próximo dos 30.000 dólares.

Depois de fazer um palpite de 23.000 dólares, actualizou a distribuição prévia.

Com base na combinação do anterior e do seu palpite, espera agora que o preço real seja de cerca de 26.000 dólares.

**Exercício:** Agora suponha que é o Jogador 2.  Quando vê a sua vitrina, supõe que o preço total é de 38.000 dólares.

Utilize `diff2` para construir uma distribuição normal que represente a distribuição dos seus erros de estimativa.

Calcule a probabilidade do seu palpite para cada preço real e utilize-o para actualizar o `prior2`.

Traçar a distribuição posterior e calcular a média posterior.  Com base no anterior e no seu palpite, qual espera que seja o preço real da vitrina?

In [26]:
# Solution goes here

In [27]:
# Solution goes here

In [28]:
# Solution goes here

In [29]:
# Solution goes here

In [30]:
# Solution goes here

In [31]:
# Solution goes here

## Probabilidade de ganhar

Agora que temos uma distribuição posterior para cada jogador, vamos pensar na estratégia.

Em primeiro lugar, do ponto de vista do Jogador 1, vamos calcular a probabilidade de o Jogador 2 fazer ofertas excessivas.  Para o manter simples, vou utilizar apenas o desempenho de jogadores passados, ignorando o valor da vitrine. 

A função que se segue toma uma sequência de licitações passadas e devolve a fracção que sobreproveu.

In [32]:
def prob_overbid(sample_diff):
    """Compute the probability of an overbid."""
    return np.mean(sample_diff > 0)

Aqui está uma estimativa para a probabilidade de o Jogador 2 fazer ofertas excessivas.

In [33]:
prob_overbid(sample_diff2)

Agora suponhamos que o Player 1 dá menos de 5.000 dólares.

Qual é a probabilidade de o Jogador 2 não dar mais?

A função seguinte utiliza o desempenho passado para estimar a probabilidade de um jogador subestimar em mais do que um determinado montante, `diff`:

In [34]:
def prob_worse_than(diff, sample_diff):
    """Probability opponent diff is worse than given diff."""
    return np.mean(sample_diff < diff)

Aqui está a probabilidade de o Jogador 2 dar menos de 5.000$.

In [35]:
prob_worse_than(-5000, sample_diff2)

E aqui está a probabilidade de subirem a proposta em mais de 10.000 dólares.

In [36]:
prob_worse_than(-10000, sample_diff2)

Podemos combinar estas funções para calcular a probabilidade de o Jogador 1 ganhar, dada a diferença entre a sua oferta e o preço real:

In [37]:
def compute_prob_win(diff, sample_diff):
    """Probability of winning for a given diff."""
    # if you overbid you lose
    if diff > 0:
        return 0
    
    # if the opponent overbids, you win
    p1 = prob_overbid(sample_diff)
    
    # or of their bid is worse than yours, you win
    p2 = prob_worse_than(diff, sample_diff)
    
    # p1 and p2 are mutually exclusive, so we can add them
    return p1 + p2

Aqui está a probabilidade de ganhar, dado que a sua proposta é inferior a 5.000 dólares.

In [38]:
compute_prob_win(-5000, sample_diff2)

Agora vamos olhar para a probabilidade de ganhar para uma série de possíveis diferenças.

In [39]:
xs = np.linspace(-30000, 5000, 121)
ys = [compute_prob_win(x, sample_diff2) 
      for x in xs]

Aqui está o que parece:

In [40]:
import matplotlib.pyplot as plt

plt.plot(xs, ys)

decorate(xlabel='Difference between bid and actual price ($)',
         ylabel='Probability of winning',
         title='Player 1')

Se a sua proposta for inferior a 30.000 dólares, a hipótese de ganhar é de cerca de 30%, o que é sobretudo a hipótese do seu oponente licitar mais do que o seu oponente.

À medida que as suas licitações se aproximam do preço real, a sua hipótese de ganhar aproxima-se 1.

E, claro, se fizer uma oferta excessiva, perde (mesmo que o seu adversário também faça ofertas excessivas).

**Exercício:** Executar a mesma análise do ponto de vista do Jogador 2.  Utilizando a amostra de diferenças do Jogador 1, calcular:

1. A probabilidade de o Jogador 1 fazer ofertas excessivas.

2. A probabilidade de que o Jogador 1 lance abaixo de mais de 5.000$.

3. A probabilidade de o Jogador 2 ganhar, dado que subiu a sua proposta em 5.000 dólares.

Depois traçar a probabilidade de o Jogador 2 ganhar por uma gama de possíveis diferenças entre a sua oferta e o preço real.

In [41]:
# Solution goes here

In [42]:
# Solution goes here

In [43]:
# Solution goes here

In [44]:
# Solution goes here

In [45]:
# Solution goes here

## Análise de Decisão

Na secção anterior calculámos a probabilidade de ganhar, dado que temos uma proposta inferior a um determinado montante.

Na realidade, os concorrentes não sabem quanto é que têm subproposta, porque não sabem o preço real.

Mas têm uma distribuição posterior que representa as suas crenças sobre o preço real, e podem utilizá-la para estimar a sua probabilidade de ganhar com uma determinada oferta.

A seguinte função toma uma possível oferta, uma distribuição posterior dos preços reais, e uma amostra de diferenças para o adversário.

Ele percorre os preços hipotéticos na distribuição posterior e, para cada preço,

1. Computa a diferença entre a oferta e o preço hipotético,

2. Compromete a probabilidade de o jogador ganhar, dada essa diferença, e

3. Acrescenta a soma ponderada das probabilidades, onde os pesos são as probabilidades na distribuição posterior. 

In [46]:
def total_prob_win(bid, posterior, sample_diff):
    """Computes the total probability of winning with a given bid.

    bid: your bid
    posterior: Pmf of showcase value
    sample_diff: sequence of differences for the opponent
    
    returns: probability of winning
    """
    total = 0
    for price, prob in posterior.items():
        diff = bid - price
        total += prob * compute_prob_win(diff, sample_diff)
    return total

Este laço implementa a lei da probabilidade total:

$$P(win) = ^sum_{preço} P(price) ~ P(win ~|~ price)$$

Eis a probabilidade de o Jogador 1 ganhar, com base numa licitação de 25.000 dólares e na distribuição posterior `posterior1`.

In [47]:
total_prob_win(25000, posterior1, sample_diff2)

Agora podemos fazer um loop através de uma série de possíveis lances e calcular a probabilidade de ganhar para cada um deles.

In [48]:
bids = posterior1.qs

probs = [total_prob_win(bid, posterior1, sample_diff2) 
         for bid in bids]

prob_win_series = pd.Series(probs, index=bids)

Aqui estão os resultados.

In [49]:
prob_win_series.plot(label='Player 1', color='C1')

decorate(xlabel='Bid ($)',
         ylabel='Probability of winning',
         title='Optimal bid: probability of winning')

E aqui está a oferta que maximiza as hipóteses de vitória do Jogador 1.

In [50]:
prob_win_series.idxmax()

In [51]:
prob_win_series.max()

Recorde que o seu palpite foi de 23.000 dólares.

Usando o seu palpite para calcular a distribuição posterior, a média posterior é de cerca de 26.000 dólares.

Mas a oferta que maximiza as suas hipóteses de ganhar é de 21.000 dólares.

**Exercício:** Fazer a mesma análise para o Jogador 2.

In [52]:
# Solution goes here

In [53]:
# Solution goes here

In [54]:
# Solution goes here

In [55]:
# Solution goes here

## Maximizando o Ganho Esperado

Na secção anterior calculámos a licitação que maximiza as suas hipóteses de ganhar.

E se esse é o seu objectivo, a proposta que calculámos é a melhor.

Mas vencer não é tudo.

Lembre-se que se a sua licitação for cancelada por 250 dólares ou menos, ganha ambos os expositores.

Portanto, pode ser uma boa ideia aumentar um pouco a sua proposta: aumenta a possibilidade de apresentar uma proposta excessiva e de perder, mas também aumenta a possibilidade de ganhar ambos os concursos.

Vamos ver como isso funciona.

A função seguinte calcula quanto ganhará, em média, dado o seu lance, o preço real, e uma amostra de erros para o seu oponente.

In [56]:
def compute_gain(bid, price, sample_diff):
    """Compute expected gain given a bid and actual price."""
    diff = bid - price
    prob = compute_prob_win(diff, sample_diff)

    # if you are within 250 dollars, you win both showcases
    if -250 <= diff <= 0:
        return 2 * price * prob
    else:
        return price * prob

Por exemplo, se o preço real for 35.000 dólares 

e licita {\i1}30000 dólares, 

ganhará cerca de 23.600 dólares de prémios em média, tendo em conta a sua probabilidade de perder, ganhar uma vitrina, ou ganhar ambas.

In [57]:
compute_gain(30000, 35000, sample_diff2)

Na realidade não sabemos o preço real, mas temos uma distribuição posterior que representa o que sabemos sobre ele.

Ao calcularmos a média dos preços e probabilidades na distribuição posterior, podemos calcular o ganho esperado para uma determinada oferta.

Neste contexto, "esperado" significa a média sobre os possíveis valores da vitrina, ponderada pelas suas probabilidades.

In [58]:
def expected_gain(bid, posterior, sample_diff):
    """Compute the expected gain of a given bid."""
    total = 0
    for price, prob in posterior.items():
        total += prob * compute_gain(bid, price, sample_diff)
    return total

Para o posterior calculámos anteriormente, com base num palpite de 23.000 dólares, o ganho esperado para um lance de 21.000 dólares é de cerca de 16.900 dólares.

In [59]:
expected_gain(21000, posterior1, sample_diff2)

Mas será que podemos fazer melhor? 

Para descobrir, podemos percorrer uma série de licitações e encontrar a que maximiza o ganho esperado.

In [60]:
bids = posterior1.qs

gains = [expected_gain(bid, posterior1, sample_diff2) for bid in bids]

expected_gain_series = pd.Series(gains, index=bids)

Aqui estão os resultados.

In [61]:
expected_gain_series.plot(label='Player 1', color='C2')

decorate(xlabel='Bid ($)',
         ylabel='Expected gain ($)',
         title='Optimal bid: expected gain')

Aqui está a melhor oferta.

In [62]:
expected_gain_series.idxmax()

Com esse lance, o ganho esperado é de cerca de 17.400 dólares.

In [63]:
expected_gain_series.max()

Recorde que o seu palpite inicial era de 23.000 dólares.

A oferta que maximiza a hipótese de ganhar é de 21.000 dólares.

E a licitação que maximiza o seu ganho esperado é de 22.000 dólares.

**Exercício:** Fazer a mesma análise para o Jogador 2.

In [64]:
# Solution goes here

In [65]:
# Solution goes here

In [66]:
# Solution goes here

In [67]:
# Solution goes here

## Resumo

Há muita coisa a acontecer neste capítulo, por isso vamos rever os passos:

1. Primeiro utilizámos o KDE e dados de exposições anteriores para estimar distribuições prévias para os valores das vitrinas.

2. Depois utilizámos propostas de espectáculos passados para modelar a distribuição de erros como uma distribuição normal.

3. Fizemos uma actualização Bayesiana utilizando a distribuição de erros para calcular a probabilidade dos dados.

4. Utilizámos a distribuição posterior para o valor da montra para calcular a probabilidade de ganhar para cada proposta possível, e identificámos a proposta que maximiza a probabilidade de ganhar.

5. Finalmente, utilizámos a probabilidade de ganhar para calcular o ganho esperado para cada proposta possível, e identificámos a proposta que maximiza o ganho esperado.

A propósito, este exemplo demonstra o perigo de utilizar a palavra "óptimo" sem especificar o que se está a optimizar.

A oferta que maximiza a hipótese de ganhar não é geralmente a mesma que a oferta que maximiza o ganho esperado.

## Discussão

Quando as pessoas discutem os prós e os contras da estimativa Bayesiana, em contraste com os métodos clássicos por vezes chamados "frequententist", afirmam frequentemente que em muitos casos os métodos Bayesianos e os métodos frequententistes produzem os mesmos resultados.

Na minha opinião, esta alegação está errada porque o método Bayesiano e frequentista produzem diferentes *tipos* de resultados:

* O resultado dos métodos frequentistas é geralmente um único valor que é considerado como a melhor estimativa (por um de vários critérios) ou um intervalo que quantifica a precisão da estimativa.

* O resultado dos métodos Bayesianos é uma distribuição posterior que representa todos os resultados possíveis e as suas probabilidades.

Concedida, pode utilizar a distribuição posterior para escolher uma estimativa "melhor" ou calcular um intervalo.

E nesse caso o resultado pode ser o mesmo que a estimativa do frequentador.

Mas ao fazê-lo descarta informação útil e, na minha opinião, elimina o benefício primário dos métodos Bayesianos: a distribuição posterior é mais útil do que uma única estimativa, ou mesmo um intervalo.

O exemplo neste capítulo demonstra o ponto.

Usando toda a distribuição posterior, podemos calcular a oferta que maximiza a probabilidade de ganhar, ou a oferta que maximiza o ganho esperado, mesmo que as regras para calcular o ganho sejam complicadas (e não lineares).

Com uma única estimativa ou um intervalo, não podemos fazer isso, mesmo que sejam "óptimos" em algum sentido.

Em geral, a estimativa dos frequentadores fornece pouca orientação para a tomada de decisões.

Se ouvir alguém dizer que os métodos Bayesianos e frequentistas produzem os mesmos resultados, pode estar confiante de que não compreendem os métodos Bayesianos.

## Exercícios

**Exercício:** Quando trabalhei em Cambridge, Massachusetts, geralmente apanhei o metro para a Estação Sul e depois um comboio de regresso a casa para Needham.  Como o metro era imprevisível, deixei o escritório suficientemente cedo para poder esperar até 15 minutos e ainda apanhar o comboio pendular.

Quando cheguei à paragem do metro, havia normalmente cerca de 10 pessoas à espera na plataforma.  Se houvesse menos do que isso, imaginei que tinha acabado de perder um comboio, por isso esperava esperar um pouco mais do que o habitual.  E se houvesse mais do que isso, esperava outro comboio em breve.

Mas se houvesse um *lote* mais de 10 passageiros à espera, inferi que algo estava errado, e esperava uma longa espera.  Nesse caso, poderia partir e apanhar um táxi.

Podemos utilizar a análise de decisão Bayesiana para quantificar a análise que fiz intuitivamente.  Tendo em conta o número de passageiros na plataforma, quanto tempo devemos esperar?  E quando devemos desistir e apanhar um táxi?

A minha análise deste problema está em `redline.ipynb`, que está no repositório deste livro. [Click here to run this notebook on Colab](https://colab.research.google.com/github/AllenDowney/ThinkBayes2/blob/master/notebooks/redline.ipynb).

**Exercício:** Este exercício é inspirado por uma história verdadeira.  Em 2001 criei [Green Tea Press](https://greenteapress.com) para publicar os meus livros, começando por *Pense Python*.  Encomendei 100 exemplares de uma impressora de tiragem curta e disponibilizei o livro para venda através de um distribuidor.  

Após a primeira semana, o distribuidor informou que foram vendidos 12 exemplares.  Com base nesse relatório, pensei que ficaria sem cópias em cerca de 8 semanas, pelo que me preparei para encomendar mais.  A minha impressora ofereceu-me um desconto se encomendasse mais de 1000 cópias, por isso fiquei um pouco louco e encomendei 2000.  

Alguns dias mais tarde, a minha mãe telefonou-me para me dizer que as suas *cópias* do livro tinham chegado.  Surpreendida, perguntei quantas.  Ela respondeu dez.

Afinal, tinha vendido apenas dois livros a não-relativos.  E levou muito mais tempo do que eu esperava para vender 2000 exemplares.

Os detalhes desta história são únicos, mas o problema geral é algo que quase todos os retalhistas têm de resolver.  Com base nas vendas passadas, como se prevêem as vendas futuras?  E com base nessas previsões, como é que decide quanto encomendar e quando?

Muitas vezes o custo de uma má decisão é complicado.  Se fizer muitas encomendas pequenas em vez de uma grande, é provável que os seus custos sejam mais elevados.  Se ficar sem inventário, poderá perder clientes.  E se encomendar demasiado, terá de pagar os vários custos de manutenção do inventário.

Portanto, vamos resolver uma versão do problema que enfrentei.  Será necessário algum trabalho para montar o problema; os detalhes estão no caderno para este capítulo.

Suponha que começa a vender livros em linha.  Durante a primeira semana vende 10 exemplares (e vamos supor que nenhum dos clientes é a sua mãe).  Durante a segunda semana, vende 9 exemplares.

Assumindo que a chegada de encomendas é um processo Poisson, podemos pensar nas encomendas semanais como amostras de uma distribuição Poisson com uma taxa desconhecida.

Podemos utilizar encomendas das últimas semanas para estimar o parâmetro desta distribuição, gerar uma distribuição preditiva para as semanas futuras, e calcular o tamanho da encomenda que maximizou o lucro esperado.

* Suponha que o custo de impressão do livro é de 5 dólares por exemplar, 

* Mas se encomendar 100 ou mais, é 4,50 dólares por cópia.

* Por cada livro que vende, recebe 10 dólares.

* Mas se ficar sem livros antes do fim de 8 semanas, perde 50 dólares em vendas futuras por cada semana em que fica sem stock.

* Se sobrar livros no final de 8 semanas, perde 2 dólares em custos de inventário por livro extra.

Por exemplo, suponha que recebe encomendas de 10 livros por semana, todas as semanas. Se encomendar 60 livros, 

* O custo total é de 300 dólares.  

* Vende todos os 60 livros, por isso ganha 600 dólares.  

* Mas o livro está esgotado durante duas semanas, por isso perde-se 100$ nas vendas futuras.

No total, o seu lucro é de 200 dólares.

Se encomendar 100 livros,

* O custo total é de $450.

* Vende 80 livros, por isso ganha 800 dólares.

* Mas ainda lhe restam 20 livros no final, por isso perde 40$.

No total, o seu lucro é de 310 dólares.

Combinando estes custos com a sua distribuição preditiva, quantos livros deve encomendar para maximizar o seu lucro esperado?

Para começar, as seguintes funções computam os lucros e custos de acordo com a especificação do problema:

In [68]:
def print_cost(printed):
    """Compute print costs.
    
    printed: integer number printed
    """
    if printed < 100:
        return printed * 5
    else:
        return printed * 4.5

In [69]:
def total_income(printed, orders):
    """Compute income.
    
    printed: integer number printed
    orders: sequence of integer number of books ordered
    """
    sold = min(printed, np.sum(orders))
    return sold * 10

In [70]:
def inventory_cost(printed, orders):
    """Compute inventory costs.
    
    printed: integer number printed
    orders: sequence of integer number of books ordered
    """
    excess = printed - np.sum(orders)
    if excess > 0:
        return excess * 2
    else:
        return 0

In [71]:
def out_of_stock_cost(printed, orders):
    """Compute out of stock costs.
    
    printed: integer number printed
    orders: sequence of integer number of books ordered
    """
    weeks = len(orders)
    total_orders = np.cumsum(orders)
    for i, total in enumerate(total_orders):
        if total > printed:
            return (weeks-i) * 50
    return 0

In [72]:
def compute_profit(printed, orders):
    """Compute profit.
    
    printed: integer number printed
    orders: sequence of integer number of books ordered
    """
    return (total_income(printed, orders) -
            print_cost(printed)-
            out_of_stock_cost(printed, orders) -
            inventory_cost(printed, orders))

Para testar estas funções, suponhamos que recebemos exactamente 10 encomendas por semana durante oito semanas:

In [73]:
always_10 = [10] * 8
always_10

Se imprimir 60 livros, o seu lucro líquido é de 200 dólares, como no exemplo.

In [74]:
compute_profit(60, always_10)

Se imprimir 100 livros, o seu lucro líquido é de 310 dólares.

In [75]:
compute_profit(100, always_10)

Claro que, no contexto do problema, não se sabe quantos livros serão encomendados numa determinada semana.  Nem sequer sabe a taxa média de encomendas.  No entanto, dados os dados e algumas suposições sobre o anterior, é possível calcular a distribuição da taxa de encomendas.

Terá uma oportunidade de o fazer, mas para demonstrar a análise de decisão parte do problema, começarei com a hipótese arbitrária de que as taxas de encomenda provêm de uma distribuição gama com média 9.

Aqui está um `Pmf` que representa esta distribuição.

In [76]:
from scipy.stats import gamma

alpha = 9
qs = np.linspace(0, 25, 101)
ps = gamma.pdf(qs, alpha)
pmf = Pmf(ps, qs)
pmf.normalize()
pmf.mean()

E aqui está o que parece:

In [77]:
pmf.plot(color='C1')
decorate(xlabel=r'Book ordering rate ($\lambda$)',
        ylabel='PMF')

Agora, nós *podemos* gerar uma distribuição preditiva do número de livros encomendados numa determinada semana, mas neste exemplo temos de lidar com uma função de custos complicada.  Em particular, o `out_of_stock_cost` depende da sequência de encomendas.

Assim, em vez de gerar uma distribuição preditiva, sugiro que façamos simulações.  Vou demonstrar os passos.

Primeiro, da nossa hipotética distribuição de taxas, podemos retirar uma amostra aleatória de 1000 valores. 

In [78]:
rates = pmf.choice(1000)
np.mean(rates)

Para cada taxa possível, podemos gerar uma sequência de 8 ordens.

In [79]:
np.random.seed(17)
order_array = np.random.poisson(rates, size=(8, 1000)).transpose()
order_array[:5, :]

Cada fila deste conjunto é uma sequência hipotética de ordens com base numa taxa de ordem hipotética diferente.

Agora, se me disser quantos livros imprimiu, posso calcular os seus lucros esperados, calculados em média sobre estas 1000 sequências possíveis.

In [80]:
def compute_expected_profits(printed, order_array):
    """Compute profits averaged over a sample of orders.
    
    printed: number printed
    order_array: one row per sample, one column per week
    """
    profits = [compute_profit(printed, orders)
               for orders in order_array]
    return np.mean(profits)

Por exemplo, aqui estão os lucros esperados se encomendar 70, 80, ou 90 livros.

In [81]:
compute_expected_profits(70, order_array)

In [82]:
compute_expected_profits(80, order_array)

In [83]:
compute_expected_profits(90, order_array)

Agora, vamos varrer uma gama de valores e calcular os lucros esperados em função do número de livros que imprime.

In [84]:
printed_array = np.arange(70, 110)
t = [compute_expected_profits(printed, order_array)
                    for printed in printed_array]
expected_profits = pd.Series(t, printed_array)

In [85]:
expected_profits.plot(label='')

decorate(xlabel='Number of books printed',
         ylabel='Expected profit ($)')

Aqui está a ordem óptima e o lucro esperado.

In [86]:
expected_profits.idxmax(), expected_profits.max()

Agora é a sua vez.  Escolha um anterior que considere razoável, actualize-o com os dados que lhe são fornecidos, e depois utilize a distribuição posterior para fazer a análise que acabei de demonstrar.

In [87]:
# Solution goes here

In [88]:
# Solution goes here

In [89]:
# Solution goes here

In [90]:
# Solution goes here

In [91]:
# Solution goes here

In [92]:
# Solution goes here

In [93]:
# Solution goes here

In [94]:
# Solution goes here

In [95]:
# Solution goes here

In [96]:
# Solution goes here

In [97]:
# Solution goes here